# Demo Facial Image Retrieval

## Loading elements

In [255]:
# Utility
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import subprocess
import tkinter as tk
import random
import zipfile

# Image processing
import cv2
from PIL import Image, ImageTk

# KDTree
from sklearn.neighbors import KDTree
import joblib

# Keras
# from tensorflow import keras
from keras.preprocessing import image as kimage
from keras.applications import MobileNetV2, mobilenet_v2

In [256]:
path_model = "Models/Face Image Retrieval/mob_tree.joblib"
path_celeba = "Datasets/CelebA/celeba.zip"
path_actors = "Datasets/Demo/Face Image Retrieval/"
path_celeba_attr = "Datasets/CelebA/list_attr_celeba.csv"
path_actors_attr = "Datasets/Demo/Face Image Retrieval/list_attr_actors.csv"
path_backgrouds = "Datasets/Demo/Face Image Retrieval/Backgrounds/"
path_haarcascade = "Other/haarcascade_frontalface_default.xml"

Loading the tree.

In [257]:
tree = joblib.load(path_model)

Loading the MobileNetV2.

In [258]:
mobilenet = MobileNetV2(input_shape = (224, 224, 3), weights = 'imagenet', include_top = False, pooling = 'max')

Loading the dataframes for computing accuracy.

In [259]:
df_actors = pd.read_csv(path_actors_attr)
df_celeba = pd.read_csv(path_celeba_attr)

In [260]:
actors = [file for file in os.listdir(path_actors) if file.endswith(".png")] # actors files list
backgrounds = [file for file in os.listdir(path_backgrouds) if file.endswith(".png")] # actors files list

## Acquisition

In [261]:
root= tk.Tk()

canvas1 = tk.Canvas(root, width=400, height=300, relief='raised')
canvas1.pack()

label1 = tk.Label(root, text='Select face to retrieve')
label1.config(font=('helvetica', 16))
canvas1.create_window(200, 25, window=label1)

label2 = tk.Label(root, text='Number from 0 to 23:')
label2.config(font=('helvetica', 11))
canvas1.create_window(200, 100, window=label2)

def display_text():
   global example
   example = int(example.get())
   root.destroy

example = tk.Entry(root)
example.pack()
canvas1.create_window(200, 140, window=example)

    
button1 = tk.Button(text='Select', command=lambda: [display_text(), root.destroy()], font=('helvetica', 12, 'bold'))
canvas1.create_window(200, 180, window=button1)

root.mainloop()

In [262]:
path_actor_example = path_actors + actors[example]
im = Image.open(path_actor_example)

In [263]:
# Print selected image
root = tk.Tk()

canvas1 = tk.Canvas(root, width=400, height=250, relief='raised')
canvas1.pack()

# Root window title and dimension
root.title("Selected face")
test = ImageTk.PhotoImage(im)
label1 = tk.Label(image=test)
label1.image = test
label1.place(x=0, y=0)

button1 = tk.Button(text='Close', command=lambda: root.destroy(), font=('helvetica', 12, 'bold'))

root.mainloop()

## Processing

In order to crop the background as much as possible, we use a Haar cascade classifier to detect and crop faces.

In [264]:
# Load the cascade classifier
face_cascade = cv2.CascadeClassifier(path_haarcascade)

def crop_face(path, scaleFactor, minNeighbors):
  # Read the input image
  im = cv2.imread(path)
  # Convert the image to grayscale
  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  # Detect faces in the image
  faces = face_cascade.detectMultiScale(gray, scaleFactor, minNeighbors)
  if len(faces) == 0:
    # We may try to decrease the scaleFactor parameter
    faces = face_cascade.detectMultiScale(gray, scaleFactor - 0.1, minNeighbors)
    if len(faces) == 0:
      return (0, 0)
    elif len(faces) > 1:
      return (1, 0)
    else:
      # Draw rectangles around the faces
      for (x, y, w, h) in faces:
        face = im[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        return (2, face)
  elif len(faces) > 1:
    # We may try to increase the scaleFactor parameter
    faces = face_cascade.detectMultiScale(gray, scaleFactor + 0.1, minNeighbors)
    if len(faces) == 0:
      return (0, 0)
    elif len(faces) > 1:
      return (1, 0)
    else:
      # Draw rectangles around the faces
      for (x, y, w, h) in faces:
        face = im[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        return (2, face)
  else:
  # Draw rectangles around the faces
    for (x, y, w, h) in faces:
      face = im[y:y+h, x:x+w]
      face = cv2.resize(face, (224, 224))
      return (2, face)

We crop (by means of the function crop_face) the actor face.

In [265]:
res = crop_face(path_actor_example, 1.12, 9)

if res[0] == 2:
  im = res[1]
else:
  raise Exception('Didn\'t found a unique face in the frame')

Now, we can add a custom background to our query image (i.e. the actor), in order to make it more realistic and less biased.

In [266]:
# First of all we make the background transparent
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
im = Image.fromarray(im)
im = im.convert("RGBA")
data = im.getdata()
newData = []
# Set alpha = 0 if the pixel is white (or almost white)
for item in data:
  if item[0] >= 240 and item[1] >= 240 and item[2] >= 240:
    newData.append((255, 255, 255, 0))
  else:
    newData.append(item)
im.putdata(newData)
# Now we paste the actor into the background
bg = Image.open(path_backgrouds + backgrounds[example]) 
# Resize the background image according to the same actor image dimension
bg = bg.resize(im.size)
bg.paste(im, (0, 0), im)

## Query

We query the tree that we previously loaded. Specifically, we take the first 3 nearest-neighbors for each actor.

In [267]:
def features_func(im):
  # Convert into numpy array
  x = np.array(im)
  # Preprocessing according to MobileNetV2
  x = mobilenet_v2.preprocess_input(x)
  # Expand dimensions
  x = np.expand_dims(x, axis = 0)
  # Extract features
  feat = mobilenet.predict(x, verbose = False)
  # Return features
  return feat.flatten()

In [268]:
features_actors = features_func(bg)
features_actors = np.array(features_actors)

In [269]:
dist, ind = tree.query(features_actors.reshape(1, -1), k = 3)
ind = ind[0]
dist = dist[0]

# filenames celebrities retrieved
celeb_filenames = ['celeba/' + str(i).zfill(6) + '.jpg' for i in ind]

## Evaluation

In [270]:
# extraction actor and maches attributes
actor_attr = np.array(df_actors.iloc[example, 1:])
matches_attr = np.array(df_celeba.iloc[ind, 1:].values)

# count of common attributes
count = [np.count_nonzero(np.add(actor_attr, match)) for match in matches_attr]

# distance and accuracy
dist = [round(d, 2) for d in dist]
acc = [round((c/len(actor_attr))*100, 2) for c in count]

# print accuracies and distances
for n, (a, d) in enumerate(zip(acc, dist)):
    print(f'Accuracy for image {n}: {a}%')
    print(f'Distance for image {n}: {d}\n')

Accuracy for image 0: 72.97%
Distance for image 0: 77.15

Accuracy for image 1: 70.27%
Distance for image 1: 77.35

Accuracy for image 2: 83.78%
Distance for image 2: 77.53



In [271]:
root = tk.Tk()
root.geometry("900x700")

imgs = []
with zipfile.ZipFile(path_celeba, mode="r") as archive:
    for filename, d, a in zip(celeb_filenames, dist, acc):
        # extract images from zip
        ext = archive.open(filename)
        # load image
        imgs.append(ImageTk.PhotoImage(Image.open(ext), master=root))
        # composition
        label_text= tk.Label(root, text=f'    Accuracy: {a}%\nDistance: {d}')
        label_text.config(font=('helvetica', 14))
        label_text.grid()
        label_text["compound"] = tk.LEFT
        label_text["image"] = imgs[-1]

# print actor image
a = Image.open(path_actor_example).crop([87, 0, 311, 224])
a = a.resize([350,350])
act = ImageTk.PhotoImage(a, master=root)
lab = tk.Label(root, image=act, width=350, height=350)
lab.place(x=500, y=180)    

root.mainloop()

In [272]:
# im_celeb_1 = kimage.load_img('/content/celeb/celeba/' + str(ind[0][0]).zfill(6) + '.jpg', target_size = (224, 224))
# im_celeb_2 = kimage.load_img('/content/celeb/celeba/' + str(ind[0][1]).zfill(6) + '.jpg', target_size = (224, 224))
# im_celeb_3 = kimage.load_img('/content/celeb/celeba/' + str(ind[0][2]).zfill(6) + '.jpg', target_size = (224, 224))

# fig = plt.figure(figsize = (12, 10))

# fig.add_subplot(4, 1, 1)
# plt.imshow(im)
# plt.axis('off')
# plt.title('Actor ' + str(example))
# fig.add_subplot(4, 1, 2)
# plt.imshow(im_celeb_1)
# plt.axis('off')
# plt.title(str(acc_1) + '%')
# fig.add_subplot(4, 1, 3)
# plt.imshow(im_celeb_2)
# plt.axis('off')
# plt.title(str(acc_2) + '%')
# fig.add_subplot(4, 1, 4)
# plt.imshow(im_celeb_3)
# plt.axis('off')
# plt.title(str(acc_3) + '%')